In [195]:
# Loading all of the data files, all at 100mhz or 100 samples per second
# Thanks to Katie for her files, I needed the sitting, jogging and stairs ones,
# the others are my own

import numpy as np
import csv
import itertools

cls = [] # this is store my array of classifiers (0-4) - kinda hacky but it works

#This takes in the csv files and returns an array that can be used for machine learning
#also returns the header row which is the same for all of my files
def load_files(input_file, cls_num):
    output_array = []
    input_headers = []
    with open(input_file, 'rb') as f:
        reader = csv.reader(f)
        input_headers = reader.next() #saves the headers
        for row in reader:
            output_array.append(row)
            
    # Prior testing gave me the size of the files so I know I can skip the first 10 seconds (1000 rows)
    # and then take the next 30,000 records to get 30 - 10 second chunks
    # never forget that it's all zero indexed
    output_array = output_array[999:30999]
    
    # Now let's reduce down to 10 second chunks aka 1000 rows
    compressed = []
    n = 0
    temp = []
    cls_list =[]
    
    for row in output_array:
        temp.append(row)
        n +=1
        if n == 1000: 
            temp = np.array(temp).astype(float) #convert to a numpy float array so I can do the next line
            compressed.append(temp.mean(axis=0).tolist()) #this averages by columns and makes it a list of lists
            temp = []
            cls_list.append(cls_num)
            n = 0
            
    cls.append(cls_list)
#     compressed = list(itertools.chain.from_iterable(compressed)) # this flatens the list - which turns to not be wanted

    return compressed, input_headers
    
rawWalkValues, Labels = load_files("walking2.csv",0)
rawSittingValues, Labels = load_files("sitting.csv",1)
rawCarValues, Labels = load_files("car.csv",2)
rawJogValues, Labels = load_files("Jogging.csv",3)
rawStairsValues, Labels = load_files("Steps.csv",4)


In [198]:
# This block is just to test my data loads and various setup stuff from the block above
# print Labels
# print rawWalkValues[0]
# print cls

In [185]:
#Testing the length of the arrays, I should have 30 records after chunking into 10 second sections by / 1000
# print "walk -",len(rawWalkValues)
# print "sit -",len(rawSittingValues)
# print "car -",len(rawCarValues)
# print "jog -",len(rawJogValues)
# print "stairs -",len(rawStairsValues)


In [181]:
from sklearn.feature_selection import VarianceThreshold

# # convert to right data shape, list of lists
# X = [[] for k in range(len(rawLabels))]
# for row in rawWalkValues:
#     for col in range(len(rawLabels)):
#         X[col].append(row[col])
        
sel = VarianceThreshold(threshold=(.8*(1-.8)))
print sel.fit_transform(rawWalkValues)[0]

# So this strips out the low variance stuff but doesn't help me identify the right features


[  1.11069621e+04  -3.54022814e-01  -1.13514583e+00  -1.80799684e+00
   8.89886132e-01   1.68731766e-01  -1.16298015e+01   1.50421219e+01
   2.01558452e+01]


In [126]:
# from sklearn.datasets import load_iris
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2
# iris = load_iris()
# # X, y = rawWalkValues, rawLabels

# X = [[] for k in range(len(rawLabels))]
# for row in rawWalkValues:
#     for col in range(len(rawLabels)):
#         X[col].append(row[col])

# X = np.array(X)
# X.shape

# X_new = SelectKBest(chi2, k=2).fit_transform(X, rawLabels)
# X_new.shape

# This doesn't work for negative values which is what I have - so this selection is out

In [67]:
# So this doesn't work either since I don't have the correct data sets yet

from sklearn.feature_selection import RFE
from sklearn.svm import SVR

estimator = SVR(kernel="linear")
selector = RFE(estimator,3,step=1) # select the three most informative features

# convert to right data shape, list of lists
X = [[] for k in range(len(rawLabels))]
for row in rawWalkValues:
    for col in range(len(rawLabels)):
        X[col].append(row[col])

y = []
for i in range(16):
    y.append(i)
    
# selector = selector.fit(X[1:], rawLabels[1:])

# selector = selector.fit(X, y)

# print(RFE.support_)
# print(RFE.ranking_)

In [19]:
# selector.support_
selector.ranking_


array([1193, 1456, 1631, ...,  857,  878,  931])

In [68]:
#Let's do some feature selection - which isn't the same as extraction
#http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html#example-ensemble-plot-forest-importances-py

import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

# Build a classification task using 3 informative features
X, y = make_classification(n_samples=1000,
                           n_features=10,
                           n_informative=3,
                           n_redundant=0,
                           n_repeated=0,
                           n_classes=2,
                           random_state=0,
                           shuffle=False)

# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)

forest.fit(X, y)

# This doesn't work since my X and y are not right at this point

# convert to right data shape, list of lists
# X = [[] for k in range(len(rawLabels))]
# for row in rawWalkValues:
#     for col in range(len(rawLabels)):
#         X[col].append(row[col])
                     
# # X = np.array(X)
# forest.fit(X, rawLabels)

importances = forest.feature_importances_ #array with importances of each feature

idx = np.arange(0, X.shape[1]) #create an index array, with the number of features

features_to_keep = idx[importances > np.mean(importances)] #only keep features whose importance is greater than the mean importance
#should be about an array of size 3 (about)
print features_to_keep.shape

x_feature_selected = X[:,features_to_keep] #pull X values corresponding to the most important features

# print x_feature_selected
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

(3L,)
1. feature 0 (0.250398)
2. feature 1 (0.232397)
3. feature 2 (0.148898)
4. feature 3 (0.055363)
5. feature 8 (0.054010)
6. feature 5 (0.053878)
7. feature 6 (0.052583)
8. feature 9 (0.051020)
9. feature 7 (0.050963)
10. feature 4 (0.050489)


In [73]:
print X[0:2]
print y[0:2]

[[ 1.48006984 -1.35738004  0.46249366  0.74380015  0.59214909 -0.78646842
  -1.17647315 -1.28080671  1.66165186 -0.06794512]
 [-0.55838227  0.24923089 -0.19986189  2.36022859  0.5555456   0.43952232
   0.30627249  0.99914985 -0.96606319  2.16001311]]
[0 0]


In [ ]:
# turn the raw data into a more compressed format and split into training and testing arrays
# based on all my files being a little longer than needed I'm going to drop the first 10 seconds (1000 rows)
# take the next 24 (10 second chunks or 24000 rows) as training, and then the next 6 (6000 rows) as testing
compressed = []

n = 0
temp = 0

for row in rawWalkValues:
    temp += float(row[7]) # TODO:  What column should I use? and would it be different for different files
    n +=1
    if n == 1000:
        compressed.append(temp/n)
        temp = 0
        n = 0
    
#splitting it up into two sets
train = []
test = []

i = 0
for i in range(len(compressed)):
    if i < 24:
        train.append(compressed[1])
    elif i < 30:
        test.append(compressed[1])
    



In [50]:
print train
print test

[-0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987]
[-0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987, -0.18755640999999987]


In [ ]:
# Where the Machine Learning happens
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression

clf = tree.DecisionTreeClassifier()
  

    
# >>> from sklearn import tree
# >>> X = [[0, 0], [1, 1]] training set
# >>> Y = [0, 1] training labels
# >>> clf = tree.DecisionTreeClassifier()
# >>> clf = clf.fit(X, Y)
